# eda002

In [7]:
import os
import sys
import traceback
import gc
import random
import pickle
import pathlib
import subprocess
from dataclasses import dataclass
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
import itertools

import pandas as pd

In [2]:
@dataclass
class Cfg:
    exp_name = "eda001"
    input_dir = os.getenv('INPUT_DIR')
    output_dir = os.getenv('OUTPUT_DIR')
    prep_dir = os.getenv("PREP_DIR")
cfg = Cfg()

In [3]:
train = pd.read_csv(cfg.input_dir + "train.csv")
labels = pd.read_csv(cfg.input_dir + "train_labels.csv")
test = pd.read_csv(cfg.input_dir + "test.csv")
sub = pd.read_csv(cfg.input_dir + "sample_submission.csv")

# 質問同士の相関性

In [5]:
labels
labels["question"] = labels["session_id"].apply(lambda x: x.split("_")[1].replace("q", "")).astype(int)
labels["session_id"] = labels["session_id"].apply(lambda x: x.split("_")[0]).astype(int)

In [17]:
labels_pivot = pd.pivot_table(labels, index="session_id", columns="question", values="correct")

In [8]:
qs = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]

In [30]:
xs = []
ys = []
lifts = []
total = len(labels_pivot)
for x, y  in itertools.combinations(qs, 2):
    support_x = labels_pivot[x].sum() / total
    support_y = labels_pivot[y].sum() / total
    support_xy = len(labels_pivot[(labels_pivot[x]==1)&(labels_pivot[y]==1)]) / total
    lift = support_xy / (support_x * support_y)
    xs.append(x)
    ys.append(y)
    lifts.append(lift)

In [32]:
corr = pd.DataFrame({"x":xs, "y":ys, "lift":lifts})

In [40]:
corr.sort_values("lift", ascending=False).head(50)

,x,y,lift
69,5,13,1.221265
71,5,15,1.216041
121,10,15,1.207374
66,5,10,1.188029
119,10,13,1.166090
111,9,13,1.151427
65,5,9,1.139766
102,8,13,1.139381
113,9,15,1.133832
143,14,15,1.133767


In [12]:
a["level_group"].values[0]

'0-4'

In [13]:
train.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4


In [23]:
labels["session"] = labels["session_id"].apply(lambda x: x.split("_")[0])

In [24]:
labels["session"].value_counts()

20090312431273200    18
21080421144057320    18
21080418362488136    18
21080419064598684    18
21080419184002784    18
                     ..
21020218310048650    18
21020219021855028    18
21020219325225296    18
21020220032589476    18
22100221145014656    18
Name: session, Length: 11779, dtype: int64

In [26]:
session_ids = train["session_id"].unique().tolist()

In [27]:
train[train["session_id"]==session_ids[0]]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,20090312431273200,927,1267357,navigate_click,undefined,22,NaN,927.307255,-10.355929,838.0,335.0,NaN,NaN,tomap,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
877,20090312431273200,928,1268292,map_hover,basic,22,NaN,NaN,NaN,NaN,NaN,366.0,NaN,tomap,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
878,20090312431273200,929,1269474,map_click,undefined,22,NaN,457.523005,22.141338,443.0,316.0,NaN,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
879,20090312431273200,930,1270708,navigate_click,undefined,22,NaN,224.190321,-60.268671,404.0,337.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22


In [28]:
train[train["session_id"]==session_ids[1]]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
881,20090312433251036,0,0,cutscene_click,basic,0,NaN,-394.991405,84.685314,399.0,250.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
882,20090312433251036,1,218,person_click,basic,0,NaN,-394.991405,84.685314,399.0,250.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
883,20090312433251036,2,448,person_click,basic,0,NaN,-394.991405,84.685314,399.0,250.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
884,20090312433251036,3,702,person_click,basic,0,NaN,-394.991405,84.685314,399.0,250.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
885,20090312433251036,4,948,person_click,basic,0,NaN,-394.991405,84.685314,399.0,250.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2707,20090312433251036,1871,3764520,map_hover,basic,22,NaN,NaN,NaN,NaN,NaN,49.0,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
2708,20090312433251036,1872,3764686,map_hover,basic,22,NaN,NaN,NaN,NaN,NaN,132.0,NaN,tunic.library,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
2709,20090312433251036,1873,3764935,map_click,undefined,22,NaN,476.887585,5.834737,453.0,326.0,NaN,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
2710,20090312433251036,1874,3813349,navigate_click,undefined,22,NaN,239.341836,-17.844428,414.0,309.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22


In [29]:
train[train["session_id"]==session_ids[2]]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
2712,20090314121766812,0,0,cutscene_click,basic,0,NaN,-420.991405,66.685314,373.0,268.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
2713,20090314121766812,1,2317,person_click,basic,0,NaN,-397.991405,71.685314,396.0,263.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2714,20090314121766812,2,4682,person_click,basic,0,NaN,-399.991405,69.685314,394.0,265.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2715,20090314121766812,3,8782,person_click,basic,0,NaN,-400.991405,69.685314,393.0,265.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
2716,20090314121766812,4,10749,person_click,basic,0,NaN,-399.991405,67.685314,394.0,267.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745,20090314121766812,1078,2130172,map_hover,basic,22,NaN,NaN,NaN,NaN,NaN,100.0,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
3746,20090314121766812,1079,2130573,map_click,undefined,22,NaN,453.624385,23.786947,439.0,315.0,NaN,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
3747,20090314121766812,1080,2131805,navigate_click,undefined,22,NaN,283.281230,-29.965640,443.0,317.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
3748,20090314121766812,1081,2133005,navigate_click,undefined,22,NaN,270.625777,0.376447,498.0,306.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22


In [30]:
train[train["session_id"]==session_ids[3]]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
3750,20090314363702160,0,0,cutscene_click,basic,0,NaN,-72.991405,20.685314,721.0,314.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,NaN,NaN,NaN,0-4
3751,20090314363702160,1,914,person_click,basic,0,NaN,-209.991405,169.685314,584.0,165.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
3752,20090314363702160,2,1929,person_click,basic,0,NaN,-182.991405,169.685314,611.0,165.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
3753,20090314363702160,3,2929,person_click,basic,0,NaN,-182.991405,169.685314,611.0,165.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
3754,20090314363702160,4,4403,person_click,basic,0,NaN,-182.991405,169.685314,611.0,165.0,NaN,Meetings are BORING!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,NaN,NaN,NaN,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4811,20090314363702160,1112,4204831,map_hover,basic,22,NaN,NaN,NaN,NaN,NaN,932.0,NaN,tomap,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
4812,20090314363702160,1113,4206393,map_click,undefined,22,NaN,511.913982,-1.469938,479.0,330.0,NaN,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,13-22
4813,20090314363702160,1114,4206451,navigate_click,undefined,22,NaN,181.766079,-61.783822,376.0,338.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22
4814,20090314363702160,1115,4207180,navigate_click,undefined,22,NaN,262.802419,-17.860023,456.0,316.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,NaN,NaN,NaN,13-22


In [37]:
sorted(train["fqid"].dropna().unique().tolist())

['archivist',
 'archivist_glasses',
 'block',
 'block_0',
 'block_1',
 'block_badge',
 'block_badge_2',
 'block_magnify',
 'block_nelson',
 'block_tocollection',
 'block_tomap1',
 'block_tomap2',
 'boss',
 'businesscards',
 'businesscards.card_0.next',
 'businesscards.card_1.next',
 'businesscards.card_bingo.bingo',
 'businesscards.card_bingo.next',
 'ch3start',
 'chap1_finale',
 'chap1_finale_c',
 'chap2_finale_c',
 'chap4_finale_c',
 'coffee',
 'colorbook',
 'confrontation',
 'crane_ranger',
 'cs',
 'directory',
 'directory.closeup.archivist',
 'door_block_clean',
 'door_block_talk',
 'doorblock',
 'expert',
 'flag_girl',
 'fox',
 'glasses',
 'gramps',
 'groupconvo',
 'groupconvo_flag',
 'intro',
 'janitor',
 'journals',
 'journals.hub.topics',
 'journals.pic_0.next',
 'journals.pic_1.next',
 'journals.pic_2.bingo',
 'journals.pic_2.next',
 'journals_flag',
 'journals_flag.hub.topics',
 'journals_flag.hub.topics_old',
 'journals_flag.pic_0.bingo',
 'journals_flag.pic_0.next',
 'journ

In [17]:
sub.sort_values("session_id")

,session_id,correct,session_level
0,20090109393214576_q1,0,20090109393214576_0-4
27,20090109393214576_q10,0,20090109393214576_5-12
30,20090109393214576_q11,0,20090109393214576_5-12
33,20090109393214576_q12,0,20090109393214576_5-12
36,20090109393214576_q13,0,20090109393214576_5-12
39,20090109393214576_q14,0,20090109393214576_13-22
42,20090109393214576_q15,0,20090109393214576_13-22
45,20090109393214576_q16,0,20090109393214576_13-22
48,20090109393214576_q17,0,20090109393214576_13-22
51,20090109393214576_q18,0,20090109393214576_13-22
